<a href="https://colab.research.google.com/github/AsutoshDalei/Deep-Neural-Networks/blob/master/Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spotify's dataset**
We attempt to develop a model capable of *classifying* if a song would seem appealing based on 13 features of each song and the person's history of songs he liked and disliked previously.

Importing the required packages.

In [0]:
import numpy as np
from sklearn import preprocessing 
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

Uploading the dataset onto Google Colab

In [35]:
from google.colab import files
import io

uploaded = files.upload()

Saving spodata.csv to spodata (3).csv


We obtain the column position of 'target', and slice the dataset into inputs and outputs respecticely. We do not consider the parameters of song_title and artist, since song title are unique for each sample and artists column is highly dispersed with 1343 unique instances

In [36]:
df= pd.read_csv(io.BytesIO(uploaded['spodata.csv']))
df.head()

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [38]:
df['artist'].value_counts()

Drake                                                                                 16
Rick Ross                                                                             13
Disclosure                                                                            12
Backstreet Boys                                                                       10
WALK THE MOON                                                                         10
                                                                                      ..
Stanley Turrentine                                                                     1
Le Youth                                                                               1
Charles Mingus                                                                         1
Jan Lundgren Trio                                                                      1
David Guetta - Sébastian Ingrosso - Joachim Garraud - Chris Willis - Steve Angello     1
Name: artist, Length:

In [0]:
a=df.columns.get_loc('target')
ipall=df.iloc[:,1:a]
tarall=df.iloc[:,a]
#ipall=df.drop(['Unnamed: 0','song_title','target','artist'],axis=1)

If you do wish to consider the column Artist, we have to preprocess them, since they are categorical data. This can be done by One Hot Encoding by get_dummies or LabelEncoder.

In [0]:
#ipall=pd.get_dummies(ipall,columns=['artist'],prefix=['artist'])
#enc=preprocessing.LabelEncoder()
#ipall['artist1']=enc.fit_transform(ipall['artist'])

We currently wont be focussing on the column 'artist', and hence wont be preprocessing

In [41]:
ipall.head()
#ipall.info()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904


Scaling and shuffling the input and targets. Scaling is done by minmax_scaler, but any other scaler may be also used. 

In [0]:
scaledip=shuffle(preprocessing.minmax_scale(ipall,(0,1)))
scaledtar=shuffle(preprocessing.minmax_scale(tarall,(0,1)))

Obtaining the number of test, train and validation samples to be considered.
We then divide the input and target data into train, validation and test data accordingly

In [0]:
samples=scaledip.shape[0]
trainsam=int(samples*0.7)
valsam=int(samples*0.15)

In [0]:
trainip=scaledip[:trainsam]
traintar=scaledtar[:trainsam]

valip=scaledip[trainsam:trainsam+valsam]
valtar=scaledtar[trainsam:trainsam+valsam]

testip=scaledip[trainsam+valsam:]
testtar=scaledtar[trainsam+valsam:]

Here, we save our obtained data into tensorfriedly .npz format, and access then later.

In [0]:
np.savez('traindat',ip=trainip,op=traintar)
np.savez('valdat',ip=valip,op=valtar)
np.savez('testdat',ip=testip,op=testtar)

nptr=np.load('traindat.npz')
npval=np.load('valdat.npz')
npte=np.load('testdat.npz')


In [0]:
trip,trop=nptr['ip'].astype(np.float),nptr['op'].astype(np.int)
valip,valop=npval['ip'].astype(np.float),npval['op'].astype(np.int)
teip,teop=npte['ip'].astype(np.float),npte['op'].astype(np.int)

Developing the model. This can be modified on our choice to obtain better results.

In [0]:
model=tf.keras.Sequential([
                           tf.keras.layers.Dense(50,activation='tanh'),
                           tf.keras.layers.Dense(50,activation='tanh'),
                           tf.keras.layers.Dense(25,activation='relu'),
                           tf.keras.layers.Dense(2,activation='softmax')
                          ])


We compile the fit our model with the input data. 
We have a choice to initiate a callback, called early stopping, which would terminate training once the validation loss starts increasing. The number of times we *let the validation loss to increase*, can be modified by *patience*

In [54]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=trip,y=trop,validation_data=(valip,valop),epochs=100)

cbk=tf.keras.callbacks.EarlyStopping(patience=4)
model.fit(x=trip,y=trop,validation_data=(valip,valop),epochs=100,callbacks=cbk)

Epoch 1/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6994 - accuracy: 0.5174 - val_loss: 0.7089 - val_accuracy: 0.4834
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6986 - accuracy: 0.5004 - val_loss: 0.6948 - val_accuracy: 0.5397
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5117 - val_loss: 0.6991 - val_accuracy: 0.5000
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5372 - val_loss: 0.6905 - val_accuracy: 0.5265
Epoch 5/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6909 - accuracy: 0.5287 - val_loss: 0.6911 - val_accuracy: 0.5430
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5308 - val_loss: 0.6940 - val_accuracy: 0.5298
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.5167 - val_loss: 0.6942 - val_accuracy: 0.5364
Epoch 8/100
4

We evaluate our model to see how well does it fdo with the test data

In [49]:
model.evaluate(x=teip,y=teop)

10/10 [==============================] - 0s 1ms/step - loss: 0.7558 - accuracy: 0.4704


[0.7558482885360718, 0.47039473056793213]

We can also provide in a sample data of our own to obtain a result, once we are satisfied with our model's performance. 

In [52]:
trydata=np.array([0.0102,	0.459,	204600,	0.434,	0.021900,	2,	0.1650,	-8.795,	1	,0.4310,	150.062,	4.0,	0.286])
print(model.predict(trydata.reshape(1,13),batch_size=1))

[[1.3804897e-04 9.9986196e-01]]


The above output indicates that the song's attributes show it to be having higher chances of being liked, than being disliked.